## Functions

In [2]:
import math

class Function:
    def __init__(self, lambda_expression):
        self.lambda_expression = lambda_expression
    
    def __call__(self, t):
        return self.lambda_expression(t)

class Concat:
    def __init__(self, vlist):
        self.vlist = vlist
    
    def __call__(self, t):
        tmp = 0.0
        for v in self.vlist:
            tmp += v(t)
        return tmp
        
class LinkVariable:
    def __init__(self):
        self.Link = None
    
    def __call__(self, t):
        return self.Link(t)

class Invert:
    def __init__(self, v):
        self.v = v
    
    def __call__(self, t):
        return -1.0 * self.v(t)

class Kelvin:
    def __init__(self, V, cro, U, T):
        self.V = V      #容積[m^3]
        self.cro = cro  #[kJ/m^3・K]
        self.C = cro * 1000 * V  #熱容量[J/K]
        self.U = U
        self.T = T
    
    def __call__(self, t):
        return self.T
    
    def commit(self, t):
        self.T = self.T + self.U(t) / self.C

class NewtonCooling:
    def __init__(self, T1, T2, S,  c = 1.98, dt = 1.0):
        #c値
        #-----
        #暖房時の天井面または冷房時の床表面=2.67
        #暖房時の床表面または冷房時の天井面=0.64～0.87
        #垂直壁表面=1.98
        self.c = c
        
        self.T1 = T1
        self.T2 = T2
        self.S = S
        
        self.dt = dt
    
    def get_alpha_c(self, dT):
        return self.c * pow(dT, 0.25) 
    
    def get_U(self, t):
        dT = self.T1(t) - self.T2(t)
        alpha_c = self.get_alpha_c(dT)
        q = alpha_c * dT
        U = q * self.S
        return U
    
    def __call__(self, t):
        U = self.get_U(t) * self.dt
        return -U

class Fourier:
    def __init__(self, rambda, dx, S, T1, T2, dt = 1.0):
        self.rambda = rambda
        self.dx = dx
        self.S = S
        self.T1 = T1
        self.T2 = T2
        self.dt = dt
    
    def __call__(self, t):
        return -1.0 * self.dt * self.rambda * self.S * (self.T1(t) - self.T2(t)) / self.dx

class StefanBolzmann:
    def __init__(self, F12, T1, T2, e1=0.9, e2=0.9, dt = 1.0):
        self.F12 = F12
        self.T1 = T1
        self.T2 = T2
        self.e1 = e1
        self.e2 = e2
        self.dt = dt

    def __call__(self, t):
        return -1.0 * self.dt *  self.F12 * self.e1 * self.e2 * 5.67 * ((self.T1(t) / 100)**4 - (self.T2(t) / 100)**4);